# Recommender system

Carlos Pinto Pérez
## Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np

### Users lists:

Users who have problems accesing their data

In [3]:
bad_users = pd.read_csv('data/bad_users.csv', names=['user_name'], header=0)
print(f'Bad users shape: {bad_users.shape}')
print(f'Unique users: {len(bad_users.user_name.unique())}')
bad_users.head()

Bad users shape: (354, 1)
Unique users: 354


,user_name
0,young_kappa
1,Meew2
2,no_forehead
3,Alejandro13
4,kanno_


Clean users

In [4]:
good_users = pd.read_csv('data/good_users.csv', names=['user_name'], header=0)
print(f'Good users shape: {good_users.shape}')
print(f'Unique users: {len(good_users.user_name.unique())}')
good_users.head()

Good users shape: (11156, 1)
Unique users: 11156


,user_name
0,TheCriticsClub
1,Polyphemus
2,qrdel
3,Cobbles
4,Aja


### Mangas (items)

In [5]:
mangas = pd.read_csv('data/mangas.csv')
print(f'Mangas shape: {mangas.shape}')
print(f'Unique manga names: {len(mangas.manga_name.unique())}')
print(f'Unique manga ids: {len(mangas.manga_id.unique())}')
mangas.head()

Mangas shape: (20000, 3)
Unique manga names: 19249
Unique manga ids: 20000


,manga_id,manga_name,manga_rank
0,2,Berserk,1
1,1706,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,2
2,25,Fullmetal Alchemist,3
3,13,One Piece,4
4,1,Monster,5


In [6]:
mangas['manga_name'].value_counts()

Red                 5
Kanon               5
Kurogane            4
Clover              4
Hatsukoi            4
                   ..
Amasan              1
Momoiro Renka       1
Sekirara ni Kiss    1
YugaMira            1
Momokuri            1
Name: manga_name, Length: 19249, dtype: int64

#### Care with that!

In [7]:
mangas[mangas['manga_name'] == 'Red']

,manga_id,manga_name,manga_rank
2193,12713,Red,2194
2201,24665,Red,2202
10198,4191,Red,10199
16830,8588,Red,16831
18286,18653,Red,18287


### Reviews

Note that Myanimelist can enumerate the mangas (items) by id (1, 2, 3...), but I scraped them by sorting by the Myanimelist top manga list. Users don't have enumerated ids, so I can not get a straight way to get them all. Users are thus collected by the criteria that they are the authors of the public reviews done in the top 20.000 mangas. With that, I scraped 11.156 unique users and the list of scores that each one of them published. 

Review scores are more valuable than usual scores, but there are pretty few of them. I also scraped a 'weight' measure that is the quantity of likes each review received. This could make better insights on the data. However, for simplicity, I didn't use them.

In [8]:
reviews = pd.read_csv('data/reviews_scores.csv')
print(f'Reviews shape: {reviews.shape}')
reviews.head()

Reviews shape: (21987, 4)


,manga_id,score,user,weight
0,2,10,TheCriticsClub,1315
1,2,7,Polyphemus,771
2,2,10,qrdel,704
3,2,10,Cobbles,322
4,2,10,Aja,127


### Scores

About 630k scores from the 11k unique users obtained, making a mean of about 60 scores per user. This indicates that the users that have been chosen are really prolific.

In [9]:
scores = pd.read_csv('data/scores.csv')
print(f'Scores shape: {scores.shape}')
scores.head()

Scores shape: (633718, 3)


,manga_id,score,user
0,2,10,TheCriticsClub
1,4,10,TheCriticsClub
2,1067,4,Polyphemus
3,13,8,Polyphemus
4,682,2,Polyphemus


## Data processing

Merge items data with their scores

In [10]:
ratings = pd.merge(mangas, scores, on='manga_id')
print(f'Ratings shape: {ratings.shape}')
ratings.head()

Ratings shape: (548675, 5)


,manga_id,manga_name,manga_rank,score,user
0,2,Berserk,1,10,TheCriticsClub
1,2,Berserk,1,7,Polyphemus
2,2,Berserk,1,10,qrdel
3,2,Berserk,1,10,Aja
4,2,Berserk,1,6,Tumerking


Discard the manga names and other fields. Keep in mind that manga names can be duplicated, but their ids can not.

In [11]:
ratings = ratings[['manga_id', 'user', 'score']]
ratings.head()

,manga_id,user,score
0,2,TheCriticsClub,10
1,2,Polyphemus,7
2,2,qrdel,10
3,2,Aja,10
4,2,Tumerking,6


Count the number of scores of each manga. It is possible that there are mangas with no scores, or with only a few of them?

In [12]:
# Dict with (key, value): (manga_id, scores count)
item_scores = {}
for item in ratings.manga_id:
    if item not in item_scores.keys():
        item_scores[item] = 1
    else:
        item_scores[item] = item_scores[item] + 1

df_item_scores = pd.DataFrame(data = {'manga_id': list(item_scores.keys()), 'number_scores': list(item_scores.values())})
df_item_scores.head()

,manga_id,number_scores
0,2,1913
1,1706,532
2,25,2054
3,13,2228
4,1,981


Unique mangas with at least 20 scores:

In [13]:
sum(df_item_scores['number_scores'] > 19)

4248

In [14]:
mangas_filtered = df_item_scores[df_item_scores['number_scores'] > 19]
print(f'Mangas filtered shape: {mangas_filtered.shape}')
mangas_filtered.head()

Mangas filtered shape: (4248, 2)


,manga_id,number_scores
0,2,1913
1,1706,532
2,25,2054
3,13,2228
4,1,981


Mean scores

In [15]:
mangas_mean = ratings.groupby('manga_id').agg(np.mean)
mangas_mean.columns = ['mean_score']
mangas_mean

,mean_score
manga_id,
1,8.928644
2,9.003659
3,8.764706
4,8.618705
7,8.453782
...,...
19994,4.800000
19995,6.318182
19996,5.750000


Filter the mangas to obtain the ones with more than 19 scores. This is also important not only for accuracy but for computational load.

In [16]:
mangas_cleaned = pd.merge(mangas, mangas_filtered, on=['manga_id'], how='inner')
mangas_cleaned = pd.merge(mangas_cleaned, mangas_mean, on=['manga_id'])
print(f'Mangas cleaned shape: {mangas_cleaned.shape}')
mangas_cleaned.head()

Mangas cleaned shape: (4248, 5)


,manga_id,manga_name,manga_rank,number_scores,mean_score
0,2,Berserk,1,1913,9.003659
1,1706,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,2,532,8.969925
2,25,Fullmetal Alchemist,3,2054,9.022882
3,13,One Piece,4,2228,8.618492
4,1,Monster,5,981,8.928644


Save the filtered data

In [17]:
mangas_cleaned.to_csv('data/mangas_v2.csv', index=False)

There are still repeated names.

In [18]:
len(mangas_cleaned.manga_name.unique())

4175

Filter the scores over the cleaned mangas.

In [19]:
scores_on_mangas_v2 = pd.merge(mangas_cleaned, scores, on=['manga_id'], how='inner')
print(f'Scores cleaned shape: {scores_on_mangas_v2.shape}')
scores_on_mangas_v2.head()

Scores cleaned shape: (521678, 7)


,manga_id,manga_name,manga_rank,number_scores,mean_score,score,user
0,2,Berserk,1,1913,9.003659,10,TheCriticsClub
1,2,Berserk,1,1913,9.003659,7,Polyphemus
2,2,Berserk,1,1913,9.003659,10,qrdel
3,2,Berserk,1,1913,9.003659,10,Aja
4,2,Berserk,1,1913,9.003659,6,Tumerking


We have gone from 630k scores to 545k. Next step is filter by users: lets choose the ones that have done at least 20 ratings.

In [20]:
user_scores = {}
for user in scores_on_mangas_v2.user:
    if user not in user_scores.keys():
        user_scores[user] = 1
    else:
        user_scores[user] = user_scores[user] + 1

df_user_scores = pd.DataFrame(data = {'user': list(user_scores.keys()), 'number_scores': list(user_scores.values())})
df_user_scores.head()

,user,number_scores
0,TheCriticsClub,2
1,Polyphemus,372
2,qrdel,10
3,Aja,35
4,Tumerking,140


Number of 'cleaned' users:

In [21]:
sum(df_user_scores['number_scores'] > 19)

5175

In [22]:
users_filtered = df_user_scores[df_user_scores['number_scores'] > 19]
print(f'Users filtered shape: {users_filtered.shape}')
users_filtered.head()

Users filtered shape: (5175, 2)


,user,number_scores
1,Polyphemus,372
3,Aja,35
4,Tumerking,140
5,aindah,113
6,infinity,36


So the final dataset is...

In [23]:
scores_cleaned = pd.merge(scores_on_mangas_v2[['manga_id', 'manga_name', 'manga_rank', 'score', 'user']], 
                          users_filtered, on=['user'], how='inner')
print(f'Scores cleaned shape: {scores_cleaned.shape}')
scores_cleaned.head()

Scores cleaned shape: (484502, 6)


,manga_id,manga_name,manga_rank,score,user,number_scores
0,2,Berserk,1,7,Polyphemus,372
1,25,Fullmetal Alchemist,3,7,Polyphemus,372
2,13,One Piece,4,8,Polyphemus,372
3,1,Monster,5,4,Polyphemus,372
4,4632,Oyasumi Punpun,6,10,Polyphemus,372


The filtering can continue, but this seems a reasonable number.

In [24]:
scores_cleaned[['manga_id', 'score', 'user']].to_csv('data/scores_v2.csv', index=False)

Unique final users:

In [25]:
len(scores_cleaned.user.unique())

5175

The mean of the number of ratings done by each user:

In [26]:
scores_cleaned.shape[0]/len(scores_cleaned.user.unique())

93.62357487922705

## Pivot matrix

Load the data:

In [27]:
mangas = pd.read_csv('data/mangas_v2.csv')
print(f'Mangas shape: {mangas.shape}')
print(f'Unique mangas names: {len(mangas.manga_name.unique())}')
print(f'Unique mangas ids: {len(mangas.manga_id.unique())}')
mangas.head()

Mangas shape: (4248, 5)
Unique mangas names: 4175
Unique mangas ids: 4248


,manga_id,manga_name,manga_rank,number_scores,mean_score
0,2,Berserk,1,1913,9.003659
1,1706,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,2,532,8.969925
2,25,Fullmetal Alchemist,3,2054,9.022882
3,13,One Piece,4,2228,8.618492
4,1,Monster,5,981,8.928644


In [28]:
scores = pd.read_csv('data/scores_v2.csv')
print(f'Scores shape: {scores.shape}')
scores.head()

Scores shape: (484502, 3)


,manga_id,score,user
0,2,7,Polyphemus
1,25,7,Polyphemus
2,13,8,Polyphemus
3,1,4,Polyphemus
4,4632,10,Polyphemus


In [29]:
ratings = pd.merge(mangas, scores, on='manga_id')
print(f'Ratings shape: {ratings.shape}')
ratings.head()

Ratings shape: (484502, 7)


,manga_id,manga_name,manga_rank,number_scores,mean_score,score,user
0,2,Berserk,1,1913,9.003659,7,Polyphemus
1,2,Berserk,1,1913,9.003659,10,Aja
2,2,Berserk,1,1913,9.003659,6,Tumerking
3,2,Berserk,1,1913,9.003659,10,aindah
4,2,Berserk,1,1913,9.003659,9,infinity


In [30]:
ratings = ratings[['manga_id', 'user', 'score']]
ratings.head()

,manga_id,user,score
0,2,Polyphemus,7
1,2,Aja,10
2,2,Tumerking,6
3,2,aindah,10
4,2,infinity,9


Get the pivot matrix. It has a row per user and a column per item (manga). This could be done much more efficient if it where a sparse matrix, but in this approach I used a typical DataFrame.

In [31]:
pivot = ratings.pivot_table(index=['user'], columns=['manga_id'], values='score')
pivot

manga_id,1,2,3,4,7,8,9,10,11,12,...,19947,19952,19961,19968,19980,19981,19983,19984,19987,19995
user,,,,,,,,,,,,,,,,,,,,,
--Zora--,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ariste,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
-Alians-,NaN,7.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Anokata,10.0,NaN,10.0,NaN,10.0,NaN,NaN,NaN,7.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-BlackRabbit-,9.0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,6.0,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zombor11,10.0,10.0,10.0,NaN,10.0,NaN,NaN,NaN,9.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zonnikku,10.0,8.0,10.0,NaN,NaN,NaN,NaN,NaN,5.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zucchinichop,8.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Note that one of the most difficult steps that creating a recomender system has is the generation of this matrix. In this example this can be done in a straight way, but usually this requires an enormous memory capacity. Because of that, a serious project will require sparse arrays or matrix factorization techniques at this point.

---